# SEN4018 Project

## Dataset Description:

The Pima Indians Diabetes Databas is provided by The National Institute of Diabetes and Digestive and Kidney Diseases. This dataset is a subset of the larger dataset. In this dataset, all of the patients, are Pima Indian women who are at least 21 years old. The dataset contains 8 medical predictor factors:
<ol>
  <li>Number of times pregnant</li>
  <li>Plasma glucose concentration a 2 hours in an oral glucose tolerance test</li>
  <li>Diastolic blood pressure (mm Hg)</li>
  <li>Triceps skin fold thickness (mm)</li>
  <li>2-Hour serum insulin (mu U/ml)</li>
  <li>Body mass index (weight in kg/(height in m)^2)</li>
  <li>Diabetes pedigree function</li>
  <li>Age (years)</li>
</ol> 
And there is also the dependent variable, the "Outcome" which is either equal to 1 or 0.

## Dataset Description:

## Data Visualization:

## Data selection:

## Data Preprocessing:

## Data Transformation:

## Feature Selection:

## Dataset Splitting:

## Modeling: